In [ ]:
using ValueShapes
using ArraysOfArrays
using StatsBase 
using LinearAlgebra
using Statistics
using Distributions 
using IntervalSets
using BAT

using HCubature
using JLD
using CPUTime

using Plots
using Colors
using ColorSchemes
using LaTeXStrings

using SpecialFunctions

pyplot(size=(750,500))
line_colors = ColorSchemes.tab20b;

# Generate Distribution: 

In [ ]:
# data = hcat(randn(1000), randn(1000))

# sx, sy = 0.12, 0.15

# Scale = [sx 0; 0 sy]

# theta = 0.75*pi

# c, s = cos(theta), sin(theta)

# Rot = [c -s; s c]

# T = Scale * Rot

# data_p =(data * T)';

# sigma = cov(data_p')

# @show sigma

In [ ]:
# Correlated Normal:

sigma_1 = [0.32716446841097613 0.17276467616026275; 0.17276467616026275 0.33056237691918017]
sigma_2 = [0.15152028283087893 -0.11598742336072133; -0.11598742336072133 0.1616797732933265] #[0.1572026439007445 -0.1044956058704804; -0.1044956058704804 0.13445945463874312] 
sigma_3 = [0.01942201849281335 -0.003187584896683795; -0.003187584896683795 0.017175237584791444]

N = 2 
min_v = -10
max_v = 10
lgV = N*log(max_v-min_v); 

g(x;μ=[0, 0], sigma=sigma) = pdf(MvNormal(μ, sigma), x)

f(x) = g(x, μ=[1, 1], sigma=sigma_1) + g(x, μ=[-1, -1], sigma=sigma_1) + 0.1*g(x, μ=[1.5, -1.5], sigma=sigma_3) + 0.9*g(x, μ=[-1.3, 1.3], sigma=sigma_2)

In [ ]:
# Normal Distribution

true_param =(μ1=1, μ2=-1, σ=0.2)
min_v = -8
max_v = 8
N = 4
lgV = N*log(max_v-min_v); 


function fun(x::AbstractArray; true_param=true_param)
    tmp = prod(pdf.(Normal(true_param.μ1, true_param.σ), x))
    return tmp
end

In [ ]:
# Ring: 

fun(x::AbstractArray; a=2, b=1, n=3) = -(((x[end]^2 + x[1]^2 - a)^2)/ b)^2 - sum((((x[1:end-1].^2 .+ x[2:end].^2  .-a).^2)./b).^2 )

min_v = -5
max_v = 5
N = 2
lgV = N*log(max_v-min_v); 

## Plot Distribution: 

In [ ]:
x_range = range(-3, stop=3, length=100)
y_range = range(-3, stop=3, length=100)

z = [f([i,j]) for i in x_range, j in y_range]';

levels_quantiles = [0.2, 0.3, 0.4, 0.5, 0.7,  0.8,  0.85, 0.9, 0.95, 0.99, 0.999, 1,]
levels=quantile([z...], levels_quantiles)

contour(x_range, y_range, z; fill=true, levels=levels, fillalpha=0.2, color=:blues)

# Generate MCMC Samples: 

In [ ]:
algorithm = MetropolisHastings()

tuning = AdaptiveMetropolisTuning(
    λ = 0.5,
    α = 0.15..0.35,
    β = 1.5,
    c = 1e-4..1e2
)

convergence = BrooksGelmanConvergence(
    threshold = 1.1,
    corrected = false
)

init = MCMCInitStrategy(
    init_tries_per_chain = 8..128,
    max_nsamples_init = 250,
    max_nsteps_init = 250,
    max_time_init = 180
)

burnin = MCMCBurninStrategy(
    max_nsamples_per_cycle = 1000,
    max_nsteps_per_cycle = 10000,
    max_time_per_cycle = 250,
    max_ncycles = 200
)

HMI_Manual_Settings = BAT.HMISettings(BAT.cholesky_partial_whitening!, 
        1000, 
        1.5, 
        0.1, 
        true, 
        16, 
        true, 
        Dict("cov. weighted result" => BAT.hm_combineresults_covweighted!)
    )

log_likelihood = params -> LogDVal((log(f(params.a))))

prior = NamedTupleDist(a = [[min_v .. max_v for i in 1:N]...],)

posterior = PosteriorDensity(log_likelihood, prior);

In [ ]:
nchains = 10
nsamples = 1*10^4
max_time = 150
max_nsteps = 10 * nsamples;

In [ ]:
samples, stats = bat_sample(posterior, (nsamples, nchains), MetropolisHastings());

In [ ]:
hmi_data = BAT.HMIData(unshaped.(samples))
ahmi_ex_time = @CPUelapsed BAT.hm_integrate!(hmi_data)

ahmi_integral_run =[hmi_data.integralestimates["cov. weighted result"].final.estimate, hmi_data.integralestimates["cov. weighted result"].final.uncertainty]
log_smpl_int = log.(ahmi_integral_run) .+ lgV

In [ ]:
integral = exp.(log_smpl_int)

In [ ]:
truth = 3